<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/three_plus_word_responses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Remove posts with only 1 or 2 words. Leave response length at 20.



In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "three_plus_words"
MODEL_NAME = "three_plus_words"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 2000000
NUM_EPOCHS = 30
BATCH_SIZE = 1000
MAX_SEQUENCE_LENGTH = 20


# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

Using TensorFlow backend.


     |████████████████████████████████| 71kB 4.7MB/s 
  Found existing installation: fsspec 0.5.2
    Uninstalling fsspec-0.5.2:
      Successfully uninstalled fsspec-0.5.2


In [3]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [4]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
len(dev_df)

2292907

In [7]:
# Remove 1 and 2 word responses from training set.
# Split response string by spaces.
train_df['split_response'] = train_df['response_text'].str.split()
train_df.head()

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response
0,0,57265377,M,0,Jerry,protecting birth is not the same as protecting life. you may very well pledge to the former but ...,Roger Williams,Congress_Republican,"[protecting, birth, is, not, the, same, as, protecting, life., you, may, very, well, pledge, to,..."
1,1,57265377,M,0,Andrea,you need to protect children and leave my body to me.,Roger Williams,Congress_Republican,"[you, need, to, protect, children, and, leave, my, body, to, me.]"
2,2,57265377,M,0,Sherry,thank you,Roger Williams,Congress_Republican,"[thank, you]"
3,3,57265377,M,0,Bob,thank you roger,Roger Williams,Congress_Republican,"[thank, you, roger]"
4,4,57265377,M,0,Joy,"unwanted pregnancy is a sad and unfortunate situation for anyone to find themselves in, however,...",Roger Williams,Congress_Republican,"[unwanted, pregnancy, is, a, sad, and, unfortunate, situation, for, anyone, to, find, themselves..."


In [8]:
len(train_df)

9879016

In [9]:
# Remove the split responses that have length 1 or 2.
mask = (train_df['split_response'].str.len() > 2) 
train_df = train_df.loc[mask]
len(train_df)

8685575

In [10]:
# Remove 1 and 2 word responses from dev set.
# Split response string by spaces.
dev_df['split_response'] = dev_df['response_text'].str.split()
# Remove the split responses that have length 1 or 2.
mask2 = (dev_df['split_response'].str.len() > 2) 
dev_df = dev_df.loc[mask2]
len(dev_df)

2041945

In [11]:
len(dev_df)

2041945

In [12]:
num_responses_per_post = train_df.post_id.value_counts().reset_index()
num_responses_per_post.columns = ['post_id', 'num_responses']
too_big_posts = num_responses_per_post[num_responses_per_post.num_responses > 50]
posts_to_sample = too_big_posts.post_id.values
# this gets all the rows for posts we DON'T need to sample 
new_train_df = train_df[~train_df.post_id.isin(posts_to_sample)]
# this should be true
assert(len(too_big_posts) + new_train_df.post_id.nunique() == train_df.post_id.nunique())
too_big_post_rows = train_df[train_df.post_id.isin(posts_to_sample)]
sampled_rows = too_big_post_rows.groupby('post_id').apply(lambda x: x.sample(n=50)).reset_index(drop=True)
new_train_df = pd.concat([new_train_df, sampled_rows])
new_train_df.shape

(3721104, 9)

In [0]:
# Set seeds for reproducible results.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = new_train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [15]:
train_df.op_gender.value_counts()

M    2725461
W     995643
Name: op_gender, dtype: int64

In [16]:
dev_df.op_gender.value_counts()

M    1731288
W     310657
Name: op_gender, dtype: int64

In [0]:
###########################
# Execute this if want to do oversampling.
###########################
# from sklearn.utils import resample

# male = train_df[train_df.op_gender=='M']
# female = train_df[train_df.op_gender=='W']

# female_upsampled = resample(female,
#                           replace=True, # sample with replacement
#                           n_samples=len(male), # match number in majority class
#                           random_state=27) # reproducible results
# upsampled_train = pd.concat([male,female_upsampled])
# print(upsampled_train.op_gender.value_counts())


In [0]:
# replace `train_df` with `upsampled_train` if using oversampling.
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

# replace `train_df` with `upsampled_train` if using upsampling.
new_sentences_train = get_text_list(train_df.response_text.values) 
new_sentences_test = get_text_list(dev_df.response_text.values)

In [21]:
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 05 minutes, 31 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, padding='post', maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [24]:
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
/ [4 files][  2.1 GiB/  2.1 GiB]   22.5 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
| [5 files][  2.9 GiB/  2.9 GiB]   31.8 MiB/s                                   
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [0]:
# trying to figure out which words are empty here
# counter = 0
# empty_indexes = []
# for index, row in enumerate(embedding_matrix):
#   if sum(row) == 0:
#     empty_indexes.append(index)
#     counter += 1
#   if counter > 1000:
#     break

# for idx in empty_indexes:
#   try:
#     print(tokenizer.index_word[idx])
#   except:
#     print("No entry for {}".format(idx))


In [0]:
# hmmmm....
# nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
# nonzero_elements / vocab_size

In [27]:
embedding_matrix.shape

(546516, 100)

In [28]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           54651600  
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [32]:
try:
  time_start = time.time()

  history = model.fit(X_train, y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 3721104 samples, validate on 2000000 samples
Epoch 1/30
3721104/3721104 [==============================] - 104s 28us/sample - loss: 0.5151 - acc: 0.7661 - val_loss: 0.4417 - val_acc: 0.8417
Epoch 2/30
3721104/3721104 [==============================] - 106s 28us/sample - loss: 0.4982 - acc: 0.7741 - val_loss: 0.4282 - val_acc: 0.8456
Epoch 3/30
3721104/3721104 [==============================] - 107s 29us/sample - loss: 0.4928 - acc: 0.7765 - val_loss: 0.4352 - val_acc: 0.8442
Epoch 4/30
3721104/3721104 [==============================] - 109s 29us/sample - loss: 0.4896 - acc: 0.7780 - val_loss: 0.4420 - val_acc: 0.8399
Epoch 5/30
3721104/3721104 [==============================] - 107s 29us/sample - loss: 0.4871 - acc: 0.7790 - val_loss: 0.4511 - val_acc: 0.8342
Epoch 6/30
3721104/3721104 [==============================] - 106s 29us/sample - loss: 0.4852 - acc: 0.7798 - val_loss: 0.4353 - val_acc: 0.8442
Epoch 7/30
3721104/3721104 [==============================] - 109s 29us/samp

In [33]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.7870
Testing Accuracy:  0.8315


In [34]:
# ROC AUC
from sklearn.metrics import roc_auc_score
# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:,0]
auc = roc_auc_score(y_dev, yhat_probs)
print('ROC AUC:', auc)


ROC AUC: 0.6262237658190303


In [0]:
preds = yhat_probs # model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [39]:
wrong_preds.op_gender.value_counts()

W    274590
M     69507
Name: op_gender, dtype: int64

In [40]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.051702665840656824

In [41]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response,probs,preds
1882270,10915198,10458808,M,385893,Nvfl,natural vegetables & fruits storage pvt. ltd. is engaged in providing turnkey solutions for post...,Mark Takano,Congress_Democratic,"[natural, vegetables, &, fruits, storage, pvt., ltd., is, engaged, in, providing, turnkey, solut...",0.430077,W
1721675,10227279,32185537,M,377134,Margaret,nancy pelosi will never give up her jet.,Paul Gosar,Congress_Republican,"[nancy, pelosi, will, never, give, up, her, jet.]",0.380639,W
2163125,11304105,77234050,W,400925,Linda,"""getting gov't out of the way so business can make a profit""??.......like that has worked so wel...",Cathy McMorris Rodgers,Congress_Republican,"[""getting, gov't, out, of, the, way, so, business, can, make, a, profit""??.......like, that, has...",0.693899,M
1840917,10873845,77721801,W,384959,Steve,thank you! hang in there!!!,Anna Eshoo,Congress_Democratic,"[thank, you!, hang, in, there!!!]",0.771863,M
1955514,11004906,86569077,W,389590,Steven,liberal lynch mobs. they are everywhere!,Kirsten Gillibrand,Congress_Democratic,"[liberal, lynch, mobs., they, are, everywhere!]",0.870950,M
1989428,11038820,86569077,W,389932,Rob,is this not a state issue?,Kirsten Gillibrand,Congress_Democratic,"[is, this, not, a, state, issue?]",0.749740,M
1990841,11040233,86569077,W,389948,James,"how many of those were seized under stop and frisk? sure, the streets are orderly and clean, but...",Kirsten Gillibrand,Congress_Democratic,"[how, many, of, those, were, seized, under, stop, and, frisk?, sure,, the, streets, are, orderly...",0.739544,M
392010,3142101,23984618,W,83655,Eric,"a nice gesture, but the aha needs to be repealed. it has going to continue to be an anchor on th...",Ann Wagner,Congress_Republican,"[a, nice, gesture,, but, the, aha, needs, to, be, repealed., it, has, going, to, continue, to, b...",0.797502,M
379921,3130012,23984618,W,83220,Sue,people act so shocked at pp... really??? this has been going on since 1973... it is nothing new....,Ann Wagner,Congress_Republican,"[people, act, so, shocked, at, pp..., really???, this, has, been, going, on, since, 1973..., it,...",0.714035,M
2123686,11264666,77234050,W,399732,Amy,as an american & given his track record...i would not count on our president to do the right thi...,Cathy McMorris Rodgers,Congress_Republican,"[as, an, american, &, given, his, track, record...i, would, not, count, on, our, president, to, ...",0.896385,M


In [42]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response,probs,preds
1993131,11042523,86569077,W,389977,Colleen,you mean mr paul mark? rand opportunist paul?,Kirsten Gillibrand,Congress_Democratic,"[you, mean, mr, paul, mark?, rand, opportunist, paul?]",0.999967,M
1951556,11000948,86569077,W,389558,Tom,"thank you, sen. gillibrand. too bad about chuck.",Kirsten Gillibrand,Congress_Democratic,"[thank, you,, sen., gillibrand., too, bad, about, chuck.]",0.999930,M
2032565,11081957,86569077,W,390399,Kathy,wish i was in that district. good luck mr. weprin.,Kirsten Gillibrand,Congress_Democratic,"[wish, i, was, in, that, district., good, luck, mr., weprin.]",0.999920,M
429525,3362760,76269753,W,103748,Edwin,hi ms. anti-constitution. would you please do us all a favor and go hear rand speak? you probabl...,Dianne Feinstein,Congress_Democratic,"[hi, ms., anti-constitution., would, you, please, do, us, all, a, favor, and, go, hear, rand, sp...",0.999896,M
2024355,11073747,86569077,W,390259,Mike,"do paterson new jersey a favour, senator, support congressman pascrell in his democratic primary...",Kirsten Gillibrand,Congress_Democratic,"[do, paterson, new, jersey, a, favour,, senator,, support, congressman, pascrell, in, his, democ...",0.999846,M
2290366,13626031,4938109,W,528810,William,it was always racism from day one. anything to get back at the best president america has ever h...,Yvette Clarke,Congress_Democratic,"[it, was, always, racism, from, day, one., anything, to, get, back, at, the, best, president, am...",0.999810,M
2160154,11301134,77234050,W,400749,Becca,i am very proud to personally know justin. he really is an inspiration for all of us. great jo...,Cathy McMorris Rodgers,Congress_Republican,"[i, am, very, proud, to, personally, know, justin., he, really, is, an, inspiration, for, all, o...",0.999794,M
2050665,11100057,86569077,W,390694,Joe,senator gillibrand is the leader in transparency and openness in congress. she has made her meet...,Kirsten Gillibrand,Congress_Democratic,"[senator, gillibrand, is, the, leader, in, transparency, and, openness, in, congress., she, has,...",0.999786,M
1915222,10964614,86569077,W,389341,Paula,"thank you, senator gillibrand. new yorkers are fortunate to have you and chuck schumer represent...",Kirsten Gillibrand,Congress_Democratic,"[thank, you,, senator, gillibrand., new, yorkers, are, fortunate, to, have, you, and, chuck, sch...",0.999774,M
2103705,11244685,77234050,W,399379,Angel,thanks for not even considering the batwing crazy louie.,Cathy McMorris Rodgers,Congress_Republican,"[thanks, for, not, even, considering, the, batwing, crazy, louie.]",0.999768,M


In [43]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response,probs,preds
991634,5240688,46506154,M,161317,Laura,"keep the faith ms. ayotte, you are good for new hampshire and our country !! you have more supp...",Marco Rubio,Congress_Republican,"[keep, the, faith, ms., ayotte,, you, are, good, for, new, hampshire, and, our, country, !!, you...",4.470348e-07,W
991742,5240796,46506154,M,161317,Karen,stand strong senator ayotte! we are with you all the way!,Marco Rubio,Congress_Republican,"[stand, strong, senator, ayotte!, we, are, with, you, all, the, way!]",1.761317e-05,W
1870185,10903113,10458808,M,385776,Hannah,i see tulsi!,Mark Takano,Congress_Democratic,"[i, see, tulsi!]",1.981854e-05,W
912545,5161599,46506154,M,161121,Connie,"it was nice of him to support joni, but she already had my vote.",Marco Rubio,Congress_Republican,"[it, was, nice, of, him, to, support, joni,, but, she, already, had, my, vote.]",2.977252e-05,W
1705630,10211234,32185537,M,376624,Judi,way to go zoe mighty proud of you. signed a native arizonan :-),Paul Gosar,Congress_Republican,"[way, to, go, zoe, mighty, proud, of, you., signed, a, native, arizonan, :-)]",3.179908e-05,W
923031,5172085,46506154,M,161173,Amy,amen debra !,Marco Rubio,Congress_Republican,"[amen, debra, !]",3.620982e-05,W
1887369,10920297,10458808,M,386070,Stephen,yes yes yes. kamala. you have always been there for us!!! fantastic!,Mark Takano,Congress_Democratic,"[yes, yes, yes., kamala., you, have, always, been, there, for, us!!!, fantastic!]",4.437566e-05,W
1648690,10129549,51495713,M,370568,Pam,"i feel you are doing a great job! i too am extremely disappointed with the rest of the gop, they...",Richard Shelby,Congress_Republican,"[i, feel, you, are, doing, a, great, job!, i, too, am, extremely, disappointed, with, the, rest,...",5.093217e-05,W
887335,5136389,46506154,M,160982,Kim,absolutely joni rocks....................jan,Marco Rubio,Congress_Republican,"[absolutely, joni, rocks....................jan]",5.099177e-05,W
349253,2621876,46566554,M,67098,John,you send copy to congresswoman vicky hartzler,Sam Graves,Congress_Republican,"[you, send, copy, to, congresswoman, vicky, hartzler]",5.853176e-05,W
